In [1]:
from typing import List
import pickle
from natasha import (
    Segmenter,
    MorphVocab,
    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
    NewsNERTagger,
    NamesExtractor,
    Doc
)

In [2]:
segmenter = Segmenter()
morph_vocab = MorphVocab()
emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
syntax_parser = NewsSyntaxParser(emb)
ner_tagger = NewsNERTagger(emb)
names_extractor = NamesExtractor(morph_vocab)

Классы, которыми мы представляем тексты (мы считаем предложение за "текст"), в котором есть метод, который умеет делать разбор своего текста и выдавать список слов, которые представлены отдельным классом

In [16]:
class Word:
    token: str
    lemma: str
    pos: str

    def __init__(self, token: str, lemma: str, pos: str) -> None:
        self.token = token
        self.lemma = lemma
        self.pos = pos

    def __str__(self) -> str:
        return self.token + ' ' + self.lemma + ' ' + self.pos

    def __repr__(self) -> str:
        return self.token + ' ' + self.lemma + ' ' + self.pos

class Text:
    source: str
    text: str
    words: List[Word]

    def __init__(self, source: str, text: str) -> None:
        self.current = -1
        self.source = source
        self.text = text

    def parse_text(self) -> None:
        doc = Doc(self.text)
        doc.segment(segmenter)
        doc.tag_morph(morph_tagger)
        for t in doc.tokens:
            t.lemmatize(morph_vocab)
        self.words = [Word(t.text.lower(), t.lemma, t.pos) \
                for t in doc.tokens if t.pos != 'PUNCT']

    def iter(self):
        return self
    
    def next(self):
        self.current += 1
        if self.current < len(self.words):
            return self.words[self.current]
        self.current = -1
        raise StopIteration

    def __str__(self) -> str:
        return self.source + ' ' + self.text

    def __repr__(self) -> str:
        return self.source + ' ' + self.text

Прочитываю csv с предложениями, создаю список из объектов Text и делаю разбор предложений

In [18]:
with open("articles.csv", "r") as f:
    texts = f.readlines()
del texts[0]
for i in range(len(texts)):
    texts[i] = texts[i].split('," ')
    texts[i] = Text(texts[i][0], texts[i][1])
    texts[i].parse_text()

Функция, чтобы сериализовать список с предложениями

In [25]:
def dump_texts(texts: List[Text], file: str) -> None:
    with open(file, "wb") as f:
        # pickle.dump(texts[:30:3], f)
        pickle.dump(texts, f)

Функция, чтобы десериализовать список предложений

In [5]:
def load_texts(filename: str) -> List[Text]:
    with open(filename, "rb") as f:
        return pickle.load(f)

Это функция, которая нужна была нам, когда мы что-то добавляли в классы и надо было менять их у уже сериализованных текстов, чтобы все работало

In [17]:
def change_classes(texts: List[Text]) -> None:
    for t in texts:
        t.__class__ = Text
        for w in t.words:
            w.__class__ = Word